In [ ]:
import torch
import torchvision
import numpy as np
from torch import nn

batch_size = 256
num_workers = 4
mnist_train = torchvision.datasets.FashionMNIST(root='Datasets',train=True,download=True,transform=torchvision.transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST(root='Datasets',train=False,download=False,transform=torchvision.transforms.ToTensor())
train_iter = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size,shuffle=True,num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size,shuffle=True,num_workers=num_workers)

num_inputs = 784
num_outputs = 10

for X,y in train_iter:
    print(X.size(),y.shape[0])
    break

def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n
  

class LinearNet(nn.Module):
    def __init__(self,num_inputs,num_outputs):
        super(LinearNet,self).__init__()
        self.linear = nn.Linear(num_inputs,num_outputs)
    def forward(self,x):
        y = self.linear(x.view(x.shape[0],-1))
        return y

net = LinearNet(num_inputs,num_outputs)

torch.nn.init.normal_(net.linear.weight,mean=0,std=0.01)
torch.nn.init.constant_(net.linear.bias,val=0)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(),lr=0.1)

num_epochs = 10
for epoch in range(num_epochs):
    train_l_sum,train_acc_sum,n=0.0,0.0,0
    for X,y in train_iter:
        output = net(X)
        l = loss(output,y).sum()
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        train_l_sum += l.item()
        train_acc_sum +=(output.argmax(dim=1) == y).float().sum().item()
        n += y.shape[0] 
    test_acc = evaluate_accuracy(test_iter, net)
    print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
            % (epoch + 1, train_l_sum/n, train_acc_sum/n, test_acc))